This notebook shows optode drift processing examples

In [3]:

import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
import os

from matplotlib.dates import DateFormatter, MonthLocator
import matplotlib.dates as mdates
import matplotlib.colors as colors


# Optode with irreversible drift 

### First import the optode and CTD data

In [9]:
serial_num = '204381' # 1000m

# import optode-DO
optode_oxy_corr = pd.read_csv(r'Files\204381\204381_data.csv')
optode_oxy_corr.index = pd.to_datetime(optode_oxy_corr.index)

# Import cal-dip optode
station = 4
df = pd.read_csv(r'Files\204381\downcasts_AR46.csv')
df.CastTimeUTC = pd.to_datetime(df.CastTimeUTC)
df = df.set_index(pd.to_datetime(df.CastTimeUTC))
df = df.drop_duplicates(subset=['CastTimeUTC'])
caldip_deploy_date = df.pt.where(df.Station == station).dropna().index[0]

T_caldip_deploy = df.pt.where(df.Station == station).dropna().resample('10s').nearest()
sigma0_caldip_deploy = df.sigma0.where(df.Station == station).dropna().resample('10s').nearest()
S_caldip_deploy = df.sal1.where(df.Station == station).dropna().resample('10s').nearest()
p_caldip_ctd_deploy = df.prs.where(df.Station == station).dropna().resample('10s').nearest()
ctd_caldip_oxy_deploy = df.AA_DOcorr_umolkg.where(df.Station == station).dropna().resample('10s').nearest()
lat_caldip_deploy = df.lat.iloc[0]

# Import deployment cal-cast
station = 10
T_calib_deploy = df.pt.where(df.Station == station).dropna().resample('10s').nearest()
sigma0_calib_deploy = df.sigma0.where(df.Station == station).dropna().resample('10s').nearest()
S_calib_deploy = df.sal1.where(df.Station == station).dropna().resample('10s').nearest()
p_calib_ctd_deploy = df.prs.where(df.Station == station).dropna().resample('10s').nearest()
ctd_calib_oxy_deploy = df.AA_DOcorr_umolkg.where(df.Station == station).dropna().resample('10s').nearest()
lat_calib_deploy = df.lat.iloc[0]
calcast_deploy_date = df.pt.where(df.Station == station).dropna().index[0]

#Import recovery cal-cast

df = pd.read_csv(r'Files\204381\downcasts_AR69_01.csv')
df.CastTimeUTC = pd.to_datetime(df.CastTimeUTC)
df = df.set_index(pd.to_datetime(df.CastTimeUTC))
df = df.drop_duplicates(subset=['CastTimeUTC'])
    
calcast_recov_date = df.index[0]
T_calib_recov = df.pt.where(df.Station == recov_station_calib).dropna().resample('10s').nearest()
S_calib_recov = df.sal1.where(df.Station == recov_station_calib).dropna().resample('10s').nearest()
sigma0_calib_recov = df.sigma0.where(df.Station == recov_station_calib).dropna().resample('10s').nearest()
p_calib_ctd_recov = df.prs.where(df.Station == recov_station_calib).dropna().resample('10s').nearest()
ctd_calib_oxy_recov = df.DOcorr_umolkg.where(df.Station == recov_station_calib).dropna().resample('10s').nearest()
lat_calib_recov = df.lat.iloc[0]




In [16]:
import gsw 
df_all = pd.read_csv(r'C:\Users\bikbe\Documents\Projects\GOHSNAP\Data\GOHSNAP_DATA\summary_table.csv',encoding='latin1')

df_sn = df_all[df_all.serial_number == int(serial_num)]

filename = r'C:\Users\bikbe\Documents\GitHub\optode_processing_examples\Files\204381\RBRCONCERTO_204381_20200815_2354.txt'
df = pd.read_csv(filename).set_index('Time')
df.index = pd.to_datetime(df.index) 

df["serial_number"] = serial_num
df = df.rename(columns={"Var1_1": "TEMP"})
df = df.rename(columns={"Var1_2": "dissolved_o2_concentration"})
df = df.rename(columns={"Var1_3": "calibrated_phase"})
df["LONGITUDE"] = df_sn.LATITUDE.values[0]
df["LATITUDE"] = df_sn.LONGITUDE.values[0]
df["PSAL"] = df.align(S_caldip_deploy, axis = 0)[1].dropna()

df["DEPTH"] = 0

df["PRES"] = gsw.p_from_z(-df.DEPTH.values,df.LATITUDE.values)
df["SA"] = gsw.SA_from_SP(df.PSAL,df.PRES,df.LONGITUDE,df.LATITUDE)
df["CT"] = gsw.CT_from_t(df.SA,df.TEMP,df.PRES)
df["sigma0"] = gsw.sigma0(df.SA,df.CT)
cal = aa_cal(str(serial_num),deploy=deploy)
if cal is None:
    df["DOXY"] = np.nan
    df["O2eq"] = np.nan
else:
    df["DOXY"],df["O2eq"] = O2_SVU(df["calibrated_phase"], df["TEMP"], df["PSAL"], df["PRES"], df["LONGITUDE"], df["LATITUDE"], cal['SVUFoilCoef'], S0=cal["Salinity"])
# return df

deployment_caldip_optode_oxy = df.DOXY #umol/kg
deployment_caldip_optode_oxy = deployment_caldip_optode_oxy[p_caldip_ctd_deploy.index]#*((1000+sigma0_caldip_deploy)/1000) #convert umol/kg to umol/l

NameError: name 'aa_cal' is not defined

In [ ]:

overlap = np.in1d(np.round(optode_oxy_corr.PTEMP.values.flatten(),2),np.round(T_calib_recov,2)) #where microcat temp is also in ctd temp
overlap2 = np.in1d(np.round(T_calib_recov,2),np.round(mc_last_10days.PTEMP.values.flatten(),2)) #where ctd temp is also in microcat temp